In [20]:
import geopandas as gpd
import pandas as pd
import requests
import folium
import time

from shapely.geometry import Point

In [21]:
customer_transaction = r"C:/GD_2025/6_spatial_data_science/1_location_fnb/2_pipeline/location_intelligence_potential_sales_areas/data/row-data/1_input_datas/2_customer_trasaction/gofood_dataset_transactions.csv"
geojson_jakarta = r"C:/GD_2025/6_spatial_data_science/1_location_fnb/2_pipeline/location_intelligence_potential_sales_areas/data/vektor/jakarta.geojson"

In [22]:
# read customer transaction data
dateframe_customer_transaction = pd.read_csv(customer_transaction)
dateframe_customer_transaction_cleanned = dateframe_customer_transaction.drop(columns=["merchant_area", "display", "discount_price", "isDiscount", "description"])
display(dateframe_customer_transaction_cleanned)

# read jakarta areas data
geodataframe_jakarta = gpd.read_file(geojson_jakarta)
minx, miny, maxx, maxy = geodataframe_jakarta.total_bounds

print(f"bounding box Jakarta: {miny}, {minx}, {maxy}, {maxx})")

,merchant_name,category,product,price
0,"330 Kopi, Ciledug",Kopi/Minuman/Roti,Hot Almara Kopi (kopi Susu Gula Aren),20000.0
1,"330 Kopi, Ciledug",Kopi/Minuman/Roti,Ice Almara Kopi (kopi Susu Gula Aren),22000.0
2,"330 Kopi, Ciledug",Kopi/Minuman/Roti,Hot Millsis,20000.0
3,"330 Kopi, Ciledug",Kopi/Minuman/Roti,Ice Millsis,20000.0
4,"330 Kopi, Ciledug",Kopi/Minuman/Roti,Hot Millbro,22000.0
...,...,...,...,...
45190,"Zeger!, Sidoarjo",Minuman/Kopi,1L Hazelnut Choco Milktea,70000.0
45191,"Zeger!, Sidoarjo",Minuman/Kopi,1L Dolce Latte,80000.0
45192,"Zeger!, Sidoarjo",Minuman/Kopi,1L Chocoffe,80000.0
45193,"Zeger!, Sidoarjo",Minuman/Kopi,1L Kopi Susu Gula Aren,85000.0


bounding box Jakarta: -6.372625555999946, 106.68580472300005, -6.07410022199997, 106.973391)


In [23]:
def get_coordinates_from_osm(location):
    osm_url = "http://overpass-api.de/api/interpreter"

    query = f"""
    [out:json];
    node["name"="{location}"];
    out center;
    """

    try:
        response = requests.get(osm_url, params={'data': query})
        data = response.json()

        if "elements" in data and len(data["elements"]) > 0:
            lat = data["elements"][0]["lat"]
            lon = data["elements"][0]["lon"]
            return lat, lon
        else:
            return None, None
    except Exception as error:
        print(f"Upss, error getting datas: {error}")

In [24]:
dataframe_with_coordinate = dateframe_customer_transaction_cleanned.copy()

dataframe_with_coordinate["latitude"] = None
dataframe_with_coordinate["longitude"] = None

for index, row in dataframe_with_coordinate.iterrows():
    store_name = row["merchant_name"]
    lat, lon = get_coordinates_from_osm(store_name)

    dataframe_with_coordinate.at[index, "latitude"] = lat
    dataframe_with_coordinate.at[index, "longitude"] = lon

    time.sleep(1)

display(dataframe_with_coordinate)

KeyboardInterrupt: 